In [82]:
import pandas as pd
from collections import Counter, defaultdict
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [99]:

train = pd.read_csv('/home/sanala/Juputer try/PET/data/yor/train.tsv', sep="\t")
dev = pd.read_csv('/home/sanala/Juputer try/PET/data/hau/dev.tsv', sep="\t")
test = pd.read_csv('/home/sanala/Juputer try/PET/data/hau/test.tsv', sep="\t")

train.info()
train.head(10)
dev.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  1433 non-null   object
 1   headline  1433 non-null   object
 2   text      1433 non-null   object
 3   url       1433 non-null   object
dtypes: object(4)
memory usage: 44.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  317 non-null    object
 1   headline  317 non-null    object
 2   text      317 non-null    object
 3   url       317 non-null    object
dtypes: object(4)
memory usage: 10.0+ KB


In [100]:
#There are repeated examples in the training dataset
print('Unique comments: ', train.iloc[:,1].nunique()==train.shape[0] )
print('Null values: ', train.isnull().values.any())
     

Unique comments:  True
Null values:  False


In [101]:
train['category'].value_counts()

entertainment    350
politics         350
health           278
sports           234
religion         221
Name: category, dtype: int64

In [92]:
#Usually the headlines are sort but outliers can appear.
print('average sentence length: ', train.iloc[:,1].str.split().str.len().mean())
print('stdev sentence length: ', train.iloc[:,1].str.split().str.len().std())
print('max sentence length: ', train.iloc[:,1].str.split().str.len().max())

average sentence length:  13.895283018867925
stdev sentence length:  4.083335905851185
max sentence length:  29


In [62]:
train[train.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url
4,health,ኮሮናቫይረስ፡ አውሮፕላኖች እንዴት ነው በፀረ- ተህዋሲያን የሚፀዱት?,የኮሮናቫይረስ ወረርሽኝ መከሰቱን ተከትሎ ቀጥ ብሎ የነበረውን የአለም የ...,https://www.bbc.com/amharic/53627279
9,health,በኒው ዚላንድ ሰዎች በፍቃዳቸው እንዲሞቱ የሚደነግግ ሕግ ሊወጣ ነው,በኒው ዚላንድ በጠና የታመሙ ሰዎች በፍቃዳቸው እንዲሞቱ የሚደነግግ ሕግ ሊ...,https://www.bbc.com/amharic/54744271
18,health,የአውሮፓ ዕለታዊ የኮቪድ ሞት ካለፈው ሳምንት አንጻር 40 በመቶ ከፍ ብሏል,በአውሮፓ በየቀኑ የሚመዘገበው የሞት መጠን ካለፈው ሳምንት ጋር ሲነፃፀር ...,https://www.bbc.com/amharic/54708506
19,health,የወባ በሽታ መከላከያ ክትባት ላይ የተደረገ ሙከራ ከፍተኛ ውጤት አሳየ,በኦክስፎርድ የኒቨርሲቲ ተመራማሪዎች እና በቡርኪናፋሶ ሳይንቲስቶች በጋራ ...,https://www.bbc.com/amharic/news-56827450
26,health,በአፍሪካ የተፈራውን ያክል ድህነት ሰዎችን ለኮሮናቫይረስ አጋልጧል?,ከአፍሪካ በኮሮናቫይረስ በተያዙ ሰዎች ቁጥር ቀዳሚዋ ደቡብ አፍሪካ ናት። ...,https://www.bbc.com/amharic/news-54009884
...,...,...,...,...
1217,health,ኮሮናቫይረስ: እስራኤል የፋይዘር ክትባት ውጤታማ ሆኖ ማግኘቷን ገለፀች,ከእስራኤል የክትባት መርሃ ግብር የተገኙ መረጃዎች እንደሚያመለክቱት የፋይ...,https://www.bbc.com/amharic/news-56073530
1249,health,የዓለም ጤና ድርጅት ሥራ አስፈጻሚ ቦርድ ዶ/ር ቴድሮስን ዳግም ዕጩ አድር...,የዓለም ጤና ድርጅት ሥራ አስፈጻሚ ቦርድ ዶ/ር ቴድሮስ አድሃኖም ድርጅቱን...,https://www.bbc.com/amharic/news-60133588
1288,health,ባለሙያዎች በኮቪድ ምክንያት የጠፋን የማሽተት ስሜት ለመመለስ 'የማሽተት ...,በኮቪድ-19 መያዛቸውን ተከትሎ የማሽተት ስሜታቸውን በፍጥነት መመለስ ያል...,https://www.bbc.com/amharic/56870718
1295,health,ኮሮናቫይረስ፡ የኮሮና ክትባትን በዓለም ለማሰራጨት 8ሺህ ግዙፍ አውሮፕላኖ...,ይበለውና የኮሮና ክትባት ቢሳካ በነገታው እንወጋዋለን ማለት አይደለም፡፡ ...,https://www.bbc.com/amharic/news-54093179


In [63]:
train_nodup=train.drop_duplicates(subset=['headline'], keep='first')


In [64]:
train_nodup

,category,headline,text,url
0,sports,የስፖርት ኮከቦች እና የንግድ ምልክቶቻቸው- ከቦልት እስከ ክርስቲያኖ ሮናልዶ,የአትሌቲክሱ ዓለም ኮከብ እና ፈጣኑ ሰው ዩሴን ቦልት ከውድድር በፊት እና...,https://www.bbc.com/amharic/articles/ceknk30j2xxo
1,sports,እግር ኳስ፡ ዩናይትድ፣ አርሴናል፣ ቼልሲ . . . ምን አስበዋል?,የስፖርት ጋዜጦች ስለ እግር ኳስ ምን እያሉ ነው? በሚቀጥለው ጥር የሚከፈ...,https://www.bbc.com/amharic/news-55435608
2,business,ዓለምን ካስጨነቃት የዋጋ ንረት ተጠቃሚዎቹ እነማን ናቸው?,ከኮሮናቫይረስ ወረርሽኝ ተጽእኖ ሳያገግም የዩክሬን እና ሩሲያ ጦርነት የገ...,https://www.bbc.com/amharic/articles/crgj31l8mzzo
3,health,ኮሮናቫይረስ፡ በቫይረሱ የሞቱት የሮማኒያው ከንቲባ በምርጫ አሸነፉ,በኮሮናቫይረስ የሞቱት የሮማኒያው ከንቲባ በቅርቡ የተደረገውን ምርጫ በከፍ...,https://www.bbc.com/amharic/news-54336166
4,health,ኮሮናቫይረስ፡ አውሮፕላኖች እንዴት ነው በፀረ- ተህዋሲያን የሚፀዱት?,የኮሮናቫይረስ ወረርሽኝ መከሰቱን ተከትሎ ቀጥ ብሎ የነበረውን የአለም የ...,https://www.bbc.com/amharic/53627279
...,...,...,...,...
1306,sports,አትሌቲክስ፡ ለተሰንበት ግደይ የ5 ሺህ ሜትር የዓለም ክብረ ወሰንን ሰበረች,ኢትዮጵያዊቷ ሯጭ ለተሰንበት ግደይ በትናንትናው ዕለት የአምስት ሺህ ሜትር...,https://www.bbc.com/amharic/news-54459888
1307,sports,የሳምንቱ የመጨረሻ ቀናት የፕሪሚየር ሊግ ጨዋታዎች ግምት,የዳግማዊት ንግሥት ኤልዛቤጥን ህልፈተ ህይወት ተከትሎ ያለፈው ሳምንት የእ...,https://www.bbc.com/amharic/articles/c9rkxzv0rn2o
1308,sports,በኢትዮጵያዊቷ መስከረም የተጠነሰሰው የአፍሪካ ሴቶች እግር ኳስ ቻምፒዮንስ...,የመጀመሪያው ቶታልኤነርጂስ ካፍ የአፍሪካ የሴቶች ቻምፒዮንስ ሊግ ባለፈው ...,https://www.bbc.com/amharic/news-59166789
1309,politics,የዶናልድ ትራምፕ አዲሱ ማሕበራዊ ሚድያ ምን ዋጠው?,ባለፈው ዓመት ጥቅምት ላይ ነበር የቀድሞው የአሜሪካ ፕሬዝደንት ዶናልድ ት...,https://www.bbc.com/amharic/news-60979022


In [65]:
dev[dev.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url
10,health,የ90 ዓመቷ አዛውንት በሁለት ዓይነት የኮቪድ ዝርያዎች ሞቱ,የ90 ዓመቷ አዛውንት በተለያዩ የኮሮና ዝርያዎች ሕይወታቸው ካለፈ በኋላ ...,https://www.bbc.com/amharic/news-57794897
18,business,ሔኒከን በኮቪድ ወረርሽኝ ምክንያት ገቤኣው በመቀነሱ 8ሺህ ያህል ሠራተኞች...,ግዙፉ ቢራ አምራች ኩባንያ ሔኒከን በኮሮናቫይረስ ወረርሽኝ ምከንያት የቢራ...,https://www.bbc.com/amharic/news-55984273
77,business,ሔኒከን በኮቪድ ወረርሽኝ ምክንያት ገቤኣው በመቀነሱ 8ሺህ ያህል ሠራተኞች...,ግዙፉ ቢራ አምራች ኩባንያ ሔኒከን በኮሮናቫይረስ ወረርሽኝ ምከንያት የቢራ...,https://www.bbc.com/amharic/news-55984273
96,health,“የልብ ድካም ባጋጠመኝ ጊዜ ድመቴ ሕይወቴን አተረፈው”,አንዲት ብሪታኒያዊት ሴት የልብ ድካም ባጋጠማት ወቅት ድመቷ ከነበረችበት ...,https://www.bbc.com/amharic/articles/cyepe30yg20o
110,health,“የልብ ድካም ባጋጠመኝ ጊዜ ድመቴ ሕይወቴን አተረፈው”,አንዲት ብሪታኒያዊት ሴት የልብ ድካም ባጋጠማት ወቅት ድመቷ ከነበረችበት ...,https://www.bbc.com/amharic/articles/cyepe30yg20o
111,health,የ90 ዓመቷ አዛውንት በሁለት ዓይነት የኮቪድ ዝርያዎች ሞቱ,የ90 ዓመቷ አዛውንት በተለያዩ የኮሮና ዝርያዎች ሕይወታቸው ካለፈ በኋላ ...,https://www.bbc.com/amharic/news-57794897


In [66]:
dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
 

In [67]:
dev_nodup

,category,headline,text,url
0,sports,ዘጠኝ ግብ በተስተናገደበት ጨዋታ ማንቸስተር ሲቲ ድል ተቀዳጅቷል,በፈረንጆቹ የገና በዓል ማግስት ማንቸስተር ሲቲዎች ለዘጠነኛ ተከታታይ ጊዜ...,https://www.bbc.com/amharic/news-59799565
1,sports,ለአውሮፓ ሻምፒዮንነት ቀሪዎቹ ስምንት አገራት ፍልሚያቸውን ዛሬ ይጀምራሉ,ለአውሮፓ ዋንጫ አሸናፊነት የሚፎካከሩት ቀሪዎቹ ስምንት አገራት ፍልሚያቸው...,https://www.bbc.com/amharic/news-57693522
2,business,ከባድ ፍንዳታ ያጋጠመው የክሬሚያ ድልድይ በከፊል ተከፍቷል - ሩሲያ,በከባድ ፍንዳታ የወደመው እና ከሩሲያ ወደ ክሬሚያ የሚወስደው ብቸኛ ድልድ...,https://www.bbc.com/amharic/articles/cd1gdx1dvjeo
3,business,የአጎዋ የንግድ ዕድል መስተጓጎል በፋብሪካዎችና በሠራተኞች ላይ የፈጠረው ስጋት,የአፍሪካ የእድገትና ዕድል ድንጋጌ በምኅጻረ ቃሉ አጎዋ ተብሎ በሚታወቀው ...,https://www.bbc.com/amharic/news-59297844
4,health,የኮቪድ ህሙማንን የሚያክመው ኤካ ኮተቤ ሆስፒታል እየሞላ ነው ተባለ,የኮቪድ-19 ሕሙማንን ተቀብሎ ህክምና የሚሰጠው ኤካ ኮተቤ ሆስፒታል በኮቪ...,https://www.bbc.com/amharic/news-58671327
...,...,...,...,...
183,politics,ከእስር የተለቀቀው የጋዳፊ ልጅ ቱርክ ገባ,ሦስተኛው የጋዳፊ ልጅ ከሊቢያ እስር ቤት ከተለቀቀ በኋላ ወደ ቱርክ፣ ኢስ...,https://www.bbc.com/amharic/58459657
184,business,የምግብ ዋጋ መናርን ለመቋቋም አማራጮችን እየተገበሩ ያሉ የሦስት አገር ነ...,በዓለም ዙሪያ ተከሰተውን የኮሮናቫይረስ ወረርሽኝን ተከትሎ በተከሰተ ምጣኔ...,https://www.bbc.com/amharic/articles/cjm27vvj2yxo
185,politics,አዲሱ የደኅንነት መሥሪያ ቤቱ “የሥነ ልቦና ጦርነትን የመመከት ሥልጣን’’...,የሥነ ልቦና ጦርነት ቃሉ በራሱ እጅግ አከራካሪ አንዲሁም አተረጓጎሙም የተ...,https://www.bbc.com/amharic/articles/cnl15g4xjg4o
186,sports,ዓለም በጉጉት እየጠበቀው ያለው ወሳኙ የእንግሊዝ እና ፈረንሳይ ጨዋታ,በኳታሩ የዓለም ዋንጫ ውድድር በጉጉት ከሚጠበቁ ጨዋታዎች መካከል ቀዳሚ ሊ...,https://www.bbc.com/amharic/articles/c3g4lrz5ggko


In [68]:
#We take X examples from each label, and the rest become the unlabeled data-set.
classes = train['category'].unique()
samples_per_class=50 #5 10 20 50
# create an empty list to store the randomly selected samples
random_samples = []

# iterate through each class
for class_name in classes:
    # select all rows with the current class
    class_rows = train_nodup[train_nodup['category'] == class_name]
    # randomly select X rows from the class_rows DataFrame
    random_class_samples = class_rows.sample(samples_per_class)
    # add the randomly selected samples to the list
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_train=random_samples_df.reset_index(drop=True)
unlabeled_train = train.drop(new_train.index)
#new_train
#unlabeled_train

In [69]:
#new_train.columns = [0, 1, 2,3]
#unlabeled_train.columns = [0, 1, 2,3]

In [70]:
random_samples = []
samples_per_class=5
for class_name in classes:
    class_rows = dev_nodup[dev_nodup['category'] == class_name]
    random_class_samples = class_rows.sample(samples_per_class)
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_dev=random_samples_df.reset_index(drop=True)
#new_dev.columns = [0, 1, 2,3]

In [71]:
unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
unlabeled_train.reset_index(drop=True)

,unlabeled,category,headline,text,url
0,unlabeled,business,አውስትራሊያ ፡ የሒትለርን ቪዲዮ በመጠቀም ፌስቡክ ላይ በአለቆቹ የተሳለቀ...,ስኮት ትሬሲ ይባላል፡፡ የግዙፉ የብሪቲሽ ፔትሮሊየም ኩባንያ (ቢፒ) ሠራተ...,https://www.bbc.com/amharic/news-53732694
1,unlabeled,health,ኤርትራውያን በካንሰር ለሚሰቃይ ግለሰብ በ24 ሰዓታት ውስጥ ግማሽ ሚሊዮን...,በውጭ አገራት የሚኖሩ ኤርትራውያን እና ትውልደ ኤርትራውያን ዲያስፖራዎች ...,https://www.bbc.com/amharic/news-60041868
2,unlabeled,business,ኤለን መስክ የማይክሮሶፍቱን ቢል ጌትስን በልጦ የዓለማችን 2ኛው ቢሊየነር ሆነ,የቴክኖሎጂ ፈጣሪው ኤለን መስክ በአጭር ጊዜ ውስጥ የዓለማችን 2ኛው ቢሊየ...,https://www.bbc.com/amharic/55056387
3,unlabeled,health,እንቅልፍ በመቀነስ የሥራ ጫናን ‘የመበቀል’ ልማድ,ማ ራዎ ከጠዋቱ 3 ሰዓት እስከ ምሽቱ 3 ሰዓት፤ በሳምንት ለስድስት ቀናት...,https://www.bbc.com/amharic/news-56051981
4,unlabeled,sports,ኳታር ለዓለም ዋንጫ እንዴት ተመረጠች፣ ውድድሩ ለምን ኅዳር ላይ ሆነ?,ሚሊዮኖች. . . ቢሊዮኖችም ሊሆን ይችላል፤ በጉጉት የሚጠብቁት የዓለም እ...,https://www.bbc.com/amharic/articles/c51wglqrq46o
...,...,...,...,...,...
1106,unlabeled,sports,አትሌቲክስ፡ ለተሰንበት ግደይ የ5 ሺህ ሜትር የዓለም ክብረ ወሰንን ሰበረች,ኢትዮጵያዊቷ ሯጭ ለተሰንበት ግደይ በትናንትናው ዕለት የአምስት ሺህ ሜትር...,https://www.bbc.com/amharic/news-54459888
1107,unlabeled,sports,የሳምንቱ የመጨረሻ ቀናት የፕሪሚየር ሊግ ጨዋታዎች ግምት,የዳግማዊት ንግሥት ኤልዛቤጥን ህልፈተ ህይወት ተከትሎ ያለፈው ሳምንት የእ...,https://www.bbc.com/amharic/articles/c9rkxzv0rn2o
1108,unlabeled,sports,በኢትዮጵያዊቷ መስከረም የተጠነሰሰው የአፍሪካ ሴቶች እግር ኳስ ቻምፒዮንስ...,የመጀመሪያው ቶታልኤነርጂስ ካፍ የአፍሪካ የሴቶች ቻምፒዮንስ ሊግ ባለፈው ...,https://www.bbc.com/amharic/news-59166789
1109,unlabeled,politics,የዶናልድ ትራምፕ አዲሱ ማሕበራዊ ሚድያ ምን ዋጠው?,ባለፈው ዓመት ጥቅምት ላይ ነበር የቀድሞው የአሜሪካ ፕሬዝደንት ዶናልድ ት...,https://www.bbc.com/amharic/news-60979022


In [72]:
unlabeled_train.drop(columns='category', inplace=True)
unlabeled_train

,unlabeled,headline,text,url
200,unlabeled,አውስትራሊያ ፡ የሒትለርን ቪዲዮ በመጠቀም ፌስቡክ ላይ በአለቆቹ የተሳለቀ...,ስኮት ትሬሲ ይባላል፡፡ የግዙፉ የብሪቲሽ ፔትሮሊየም ኩባንያ (ቢፒ) ሠራተ...,https://www.bbc.com/amharic/news-53732694
201,unlabeled,ኤርትራውያን በካንሰር ለሚሰቃይ ግለሰብ በ24 ሰዓታት ውስጥ ግማሽ ሚሊዮን...,በውጭ አገራት የሚኖሩ ኤርትራውያን እና ትውልደ ኤርትራውያን ዲያስፖራዎች ...,https://www.bbc.com/amharic/news-60041868
202,unlabeled,ኤለን መስክ የማይክሮሶፍቱን ቢል ጌትስን በልጦ የዓለማችን 2ኛው ቢሊየነር ሆነ,የቴክኖሎጂ ፈጣሪው ኤለን መስክ በአጭር ጊዜ ውስጥ የዓለማችን 2ኛው ቢሊየ...,https://www.bbc.com/amharic/55056387
203,unlabeled,እንቅልፍ በመቀነስ የሥራ ጫናን ‘የመበቀል’ ልማድ,ማ ራዎ ከጠዋቱ 3 ሰዓት እስከ ምሽቱ 3 ሰዓት፤ በሳምንት ለስድስት ቀናት...,https://www.bbc.com/amharic/news-56051981
204,unlabeled,ኳታር ለዓለም ዋንጫ እንዴት ተመረጠች፣ ውድድሩ ለምን ኅዳር ላይ ሆነ?,ሚሊዮኖች. . . ቢሊዮኖችም ሊሆን ይችላል፤ በጉጉት የሚጠብቁት የዓለም እ...,https://www.bbc.com/amharic/articles/c51wglqrq46o
...,...,...,...,...
1306,unlabeled,አትሌቲክስ፡ ለተሰንበት ግደይ የ5 ሺህ ሜትር የዓለም ክብረ ወሰንን ሰበረች,ኢትዮጵያዊቷ ሯጭ ለተሰንበት ግደይ በትናንትናው ዕለት የአምስት ሺህ ሜትር...,https://www.bbc.com/amharic/news-54459888
1307,unlabeled,የሳምንቱ የመጨረሻ ቀናት የፕሪሚየር ሊግ ጨዋታዎች ግምት,የዳግማዊት ንግሥት ኤልዛቤጥን ህልፈተ ህይወት ተከትሎ ያለፈው ሳምንት የእ...,https://www.bbc.com/amharic/articles/c9rkxzv0rn2o
1308,unlabeled,በኢትዮጵያዊቷ መስከረም የተጠነሰሰው የአፍሪካ ሴቶች እግር ኳስ ቻምፒዮንስ...,የመጀመሪያው ቶታልኤነርጂስ ካፍ የአፍሪካ የሴቶች ቻምፒዮንስ ሊግ ባለፈው ...,https://www.bbc.com/amharic/news-59166789
1309,unlabeled,የዶናልድ ትራምፕ አዲሱ ማሕበራዊ ሚድያ ምን ዋጠው?,ባለፈው ዓመት ጥቅምት ላይ ነበር የቀድሞው የአሜሪካ ፕሬዝደንት ዶናልድ ት...,https://www.bbc.com/amharic/news-60979022


In [73]:

#We take X examples from each label, and the rest become the unlabeled data-set.
#Instead, we could also make the new training data-set following the proportion of examples per label.
#PVP pattern-verbalizer pair (PVP)

In [74]:
new_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/amh/50sample/train.tsv", sep="\t", index=False  ,header=False)
unlabeled_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/amh/50sample/unlabeled.tsv", sep="\t", index=False,header=False)
new_dev.to_csv("/home/sanala/Juputer try/PET/data-fsl/amh/50sample/dev.tsv", sep="\t", index=False,header=False)

### Train your own PET



In [224]:
%cd /home/sanala/Juputer try/pet/


/home/sanala/Juputer try/pet
